# VERIFICANDO SE TEM CUDA

In [ ]:
import torch
import torchvision
print("torch:", torch.__version__)
print("torchvision:", torchvision.__version__)
print("CUDA disponível:", torch.cuda.is_available())

# DATASET >> https://universe.roboflow.com/force-iaope/cars-egrgd/dataset/1#

# PRÉ-AMBULO - IMPORT LIBS

In [ ]:
from ultralytics import YOLO
from PIL import Image
import cv2
import os

from modules.helper.helper import create_video_from_images, get_framerate

In [ ]:
# # Create a new YOLO model from scratch
model = YOLO("yolo11n.yaml")

# Load a pretrained YOLO model (recommended for training)
model = YOLO("yolo11n.pt")

# Train the model using the 'coco8.yaml' dataset for 3 epochs
results = model.train(data="data.yaml", epochs=3, batch=16, device="0", workers=4)#, classes=[0, 1, 2, 3, 5, 7])

# Perform object detection on an image using the model
results = model("https://ultralytics.com/images/bus.jpg")

# Export the model to ONNX format
success = model.export(format="onnx")

In [ ]:
# Caminho para o vídeo
video_path  = "input\\Teste.mp4"

save        = "output\\detect"
os.makedirs(save, exist_ok=True)

# Perform object detection on an image using the model
results = model(source=video_path)

# Visualize the results
for i, r in enumerate(results):
    try:
        # Plot results image
        im_bgr = r.plot()  # BGR-order numpy array
        if im_bgr is None:
            print(f"Resultado {i} não contém dados válidos para plotar.")
            continue
        im_rgb = Image.fromarray(im_bgr[..., ::-1])  # RGB-order PIL image
        # Save results to disk
        r.save(filename=os.path.join(save, f"results{i}.jpg"))
    except ValueError as e:
        print(f"Erro ao processar o resultado {i}: {e}")
    except Exception as e:
        print(f"Erro inesperado ao processar o resultado {i}: {e}")

# Exemplo de uso
image_folder = save
output_video_path = "output_video.mp4"
frame_rate = get_framerate(video_path)  # Substitua pelo frame rate desejado
create_video_from_images(image_folder, output_video_path, frame_rate)